In [1]:
import streamlit as st
import pandas as pd
import numpy as np
import re
from PIL import Image
import webbrowser
import json
import pickle
import sys 
import joblib
from tqdm import tqdm 

sys.path.append('./CC/')

import chemaxon
from chemaxon import *
from compound import Compound
from compound_cacher import CompoundCacher
from rdkit.Chem import rdChemReactions as Reactions
from rdkit.Chem import Draw
from rdkit import Chem

2023-07-27 12:54:13.104 INFO    rdkit: Enabling RDKit 2020.09.1 jupyter extensions


In [2]:
def load_smiles():
    db = pd.read_csv('./data/cache_compounds_20160818.csv',
                     index_col='compound_id')
    db_smiles = db['smiles_pH7'].to_dict()
    return db_smiles

def load_molsig_rad1():
    molecular_signature_r1 = json.load(open('./data/decompose_vector_ac.json'))
    return molecular_signature_r1


def load_molsig_rad2():
    molecular_signature_r2 = json.load(
        open('./data/decompose_vector_ac_r2_py3_indent_modified_manual.json'))
    return molecular_signature_r2


def load_model():
    filename = './model/M12_model_BR.pkl'
    loaded_model = joblib.load(open(filename, 'rb'))
    return loaded_model


def load_compound_cache():
    ccache = CompoundCacher()
    return ccache


In [3]:
def count_substructures(radius, molecule):
    """Helper function for get the information of molecular signature of a
    metabolite. The relaxed signature requires the number of each substructure
    to construct a matrix for each molecule.
    Parameters
    ----------
    radius : int
        the radius is bond-distance that defines how many neighbor atoms should
        be considered in a reaction center.
    molecule : Molecule
        a molecule object create by RDkit (e.g. Chem.MolFromInchi(inchi_code)
        or Chem.MolToSmiles(smiles_code))
    Returns
    -------
    dict
        dictionary of molecular signature for a molecule,
        {smiles: molecular_signature}
    """
    m = molecule
    smi_count = dict()
    atomList = [atom for atom in m.GetAtoms()]

    for i in range(len(atomList)):
        env = Chem.FindAtomEnvironmentOfRadiusN(m, radius, i)
        atoms = set()
        for bidx in env:
            atoms.add(m.GetBondWithIdx(bidx).GetBeginAtomIdx())
            atoms.add(m.GetBondWithIdx(bidx).GetEndAtomIdx())

        # only one atom is in this environment, such as O in H2O
        if len(atoms) == 0:
            atoms = {i}

        smi = Chem.MolFragmentToSmiles(m, atomsToUse=list(atoms),
                                       bondsToUse=env, canonical=True)

        if smi in smi_count:
            smi_count[smi] = smi_count[smi] + 1
        else:
            smi_count[smi] = 1
    return smi_count


In [4]:
def decompse_novel_mets_rad1(novel_smiles, radius=1):
    decompose_vector = dict()

    for cid, smiles_pH7 in novel_smiles.items():
        mol = Chem.MolFromSmiles(smiles_pH7)
        mol = Chem.RemoveHs(mol)
        # Chem.RemoveStereochemistry(mol)
        smi_count = count_substructures(radius, mol)
        decompose_vector[cid] = smi_count
    return decompose_vector


def decompse_novel_mets_rad2(novel_smiles, radius=2):
    decompose_vector = dict()

    for cid, smiles_pH7 in novel_smiles.items():
        mol = Chem.MolFromSmiles(smiles_pH7)
        mol = Chem.RemoveHs(mol)
        # Chem.RemoveStereochemistry(mol)
        smi_count = count_substructures(radius, mol)
        decompose_vector[cid] = smi_count
    return decompose_vector


In [5]:
def parse_reaction_formula_side(s):
    """
        Parses the side formula, e.g. '2 C00001 + C00002 + 3 C00003'
        Ignores stoichiometry.

        Returns:
            The set of CIDs.
    """
    if s.strip() == "null":
        return {}

    compound_bag = {}
    for member in re.split('\s+\+\s+', s):
        tokens = member.split(None, 1)
        if len(tokens) == 0:
            continue
        if len(tokens) == 1:
            amount = 1
            key = member
        else:
            amount = float(tokens[0])
            key = tokens[1]

        compound_bag[key] = compound_bag.get(key, 0) + amount

    return compound_bag


def parse_formula(formula, arrow='<=>', rid=None):
    """
        Parses a two-sided formula such as: 2 C00001 => C00002 + C00003

        Return:
            The set of substrates, products and the direction of the reaction
    """
    tokens = formula.split(arrow)
    if len(tokens) < 2:
        print(('Reaction does not contain the arrow sign (%s): %s'
               % (arrow, formula)))
    if len(tokens) > 2:
        print(('Reaction contains more than one arrow sign (%s): %s'
               % (arrow, formula)))

    left = tokens[0].strip()
    right = tokens[1].strip()

    sparse_reaction = {}
    for cid, count in parse_reaction_formula_side(left).items():
        sparse_reaction[cid] = sparse_reaction.get(cid, 0) - count

    for cid, count in parse_reaction_formula_side(right).items():
        sparse_reaction[cid] = sparse_reaction.get(cid, 0) + count

    return sparse_reaction


In [6]:
def draw_rxn_figure(rxn_dict, db_smiles, novel_smiles):
    # db_smiles = load_smiles()

    left = ''
    right = ''

    for met, stoic in rxn_dict.items():
        if met == "C00080" or met == "C00282":
            continue  # hydogen is not considered
        if stoic > 0:
            if met in db_smiles:
                right = right + db_smiles[met] + '.'
            else:
                right = right + novel_smiles[met] + '.'
        else:
            if met in db_smiles:
                left = left + db_smiles[met] + '.'
            else:
                left = left + novel_smiles[met] + '.'
    smarts = left[:-1] + '>>' + right[:-1]
    # print smarts
    smarts = str(smarts)
    rxn = Reactions.ReactionFromSmarts(smarts, useSmiles=True)
    return Draw.ReactionToImage(rxn)  # , subImgSize=(400, 400))

In [7]:
def get_rule(rxn_dict, molsig1, molsig2, novel_decomposed1, novel_decomposed2):
    if novel_decomposed1 != None:
        for cid in novel_decomposed1:
            molsig1[cid] = novel_decomposed1[cid]
    if novel_decomposed2 != None:
        for cid in novel_decomposed2:
            molsig2[cid] = novel_decomposed2[cid]

    molsigna_df1 = pd.DataFrame.from_dict(molsig1).fillna(0)
    all_mets1 = molsigna_df1.columns.tolist()
    all_mets1.append("C00080")
    all_mets1.append("C00282")

    molsigna_df2 = pd.DataFrame.from_dict(molsig2).fillna(0)
    all_mets2 = molsigna_df2.columns.tolist()
    all_mets2.append("C00080")
    all_mets2.append("C00282")

    moieties_r1 = open('./data/group_names_r1.txt')
    moieties_r2 = open('./data/group_names_r2_py3_modified_manual.txt')
    moie_r1 = moieties_r1.read().splitlines()
    moie_r2 = moieties_r2.read().splitlines()

    molsigna_df1 = molsigna_df1.reindex(moie_r1)
    molsigna_df2 = molsigna_df2.reindex(moie_r2)

    rule_df1 = pd.DataFrame(index=molsigna_df1.index)
    rule_df2 = pd.DataFrame(index=molsigna_df2.index)
    # for rid, value in reaction_dict.items():
    #     # skip the reactions with missing metabolites
    #     mets = value.keys()
    #     flag = False
    #     for met in mets:
    #         if met not in all_mets:
    #             flag = True
    #             break
    #     if flag: continue

    rule_df1['change'] = 0
    for met, stoic in rxn_dict.items():
        if met == "C00080" or met == "C00282":
            continue  # hydogen is zero
        rule_df1['change'] += molsigna_df1[met] * stoic

    rule_df2['change'] = 0
    for met, stoic in rxn_dict.items():
        if met == "C00080" or met == "C00282":
            continue  # hydogen is zero
        rule_df2['change'] += molsigna_df2[met] * stoic

    rule_vec1 = rule_df1.to_numpy().T
    rule_vec2 = rule_df2.to_numpy().T

    m1, n1 = rule_vec1.shape
    m2, n2 = rule_vec2.shape

    zeros1 = np.zeros((m1, 44))
    zeros2 = np.zeros((m2, 44))
    X1 = np.concatenate((rule_vec1, zeros1), 1)
    X2 = np.concatenate((rule_vec2, zeros2), 1)

    rule_comb = np.concatenate((X1, X2), 1)

    # rule_df_final = {}
    # rule_df_final['rad1'] = rule_df1
    # rule_df_final['rad2'] = rule_df2
    return rule_comb, rule_df1, rule_df2

In [8]:
def get_ddG0(rxn_dict, pH, I, novel_mets):
    ccache = CompoundCacher()
    # ddG0 = get_transform_ddG0(rxn_dict, ccache, pH, I, T)
    T = 298.15
    ddG0_forward = 0
    for compound_id, coeff in rxn_dict.items():
        if novel_mets != None and compound_id in novel_mets:
            comp = novel_mets[compound_id]
        else:
            comp = ccache.get_compound(compound_id)
        ddG0_forward += coeff * comp.transform_pH7(pH, I, T)

    return ddG0_forward

In [9]:
def get_dG0(rxn_dict, rid, pH, I, loaded_model, molsig_r1, molsig_r2, novel_decomposed_r1, novel_decomposed_r2, novel_mets):

    # rule_df = get_rxn_rule(rid)
    rule_comb, rule_df1, rule_df2 = get_rule(
        rxn_dict, molsig_r1, molsig_r2, novel_decomposed_r1, novel_decomposed_r2)

    X = rule_comb

    ymean, ystd = loaded_model.predict(X, return_std=True)

    result = {}
    # result['dG0'] = ymean[0] + get_ddG0(rxn_dict, pH, I)
    # result['standard deviation'] = ystd[0]

    # result_df = pd.DataFrame([result])
    # result_df.style.hide_index()
    # return result_df
    return ymean[0] + get_ddG0(rxn_dict, pH, I, novel_mets), ystd[0], rule_df1, rule_df2
    # return ymean[0],ystd[0]


In [10]:
def parse_novel_molecule(add_info):
    result = {}
    for cid, InChI in add_info.items():
        c = Compound.from_inchi('Test', cid, InChI)
        result[cid] = c
    return result


def parse_novel_smiles(result):
    novel_smiles = {}
    for cid, c in result.items():
        smiles = c.smiles_pH7
        novel_smiles[cid] = smiles
    return novel_smiles


In [11]:
db_smiles = load_smiles()
molsig_r1 = load_molsig_rad1()
molsig_r2 = load_molsig_rad2()

loaded_model = load_model()
ccache = load_compound_cache()

## Estimating dG for reaction 

In [12]:
Rxn_str_test = "C00031 + 1.14 C00014 <=> 0.29 C00011 + 3.14 C00001"

In [13]:
Rxn_dict_test = parse_formula(Rxn_str_test)
pH = 7

In [14]:
mu_test, std_test, rule_df1_test, rule_df2_test = get_dG0(Rxn_dict_test, 'Rxn_test', pH, 0.25, loaded_model, molsig_r1, molsig_r2, [], [], [])

print("dG = %.2f ± %.2f kJ/mol" % (mu_test, std_test))


# print(mu_test)
# print(mu1_test)


dG = -268.85 ± 3.99 kJ/mol


## Bulk ModelSeed database dG generation

In [15]:
rxn_alias_fread = open('./data/Unique_ModelSEED_Reaction_Aliases.txt').readlines()

In [19]:
ModelSEED_ID = []
External_ID = []
Source = []

for i in tqdm(rxn_alias_fread):
    temp = i.split('\t')
    ModelSEED_ID.append(temp[0])
    External_ID.append(temp[1])
    Source.append(temp[2])
    
    

100%|██████████| 343680/343680 [00:00<00:00, 943467.42it/s]


In [20]:
ModelSEED_ID_header = ModelSEED_ID[0]
External_ID_header = External_ID[0]
Source_header = Source[0]

ModelSEED_ID = ModelSEED_ID[1:]
External_ID = External_ID[1:]
Source = Source[1:]


In [21]:
ModelSEED_alias_dict = {}
ModelSEED_alias_dict[ModelSEED_ID_header] = ModelSEED_ID
ModelSEED_alias_dict[External_ID_header] = External_ID
ModelSEED_alias_dict[Source_header] = Source

In [22]:
ModelSEED_alias_df = pd.DataFrame(ModelSEED_alias_dict, index=None)

In [23]:
ModelSEED_alias_df.head()

,ModelSEED ID,External ID,Source\n
0,rxn00001,R_R00004_c,AlgaGEM\n
1,rxn00001,INORGPYROPHOSPHAT-RXN,AraCyc\n
2,rxn00001,R_R00004_c,AraGEM\n
3,rxn00001,IPP1,BiGG\n
4,rxn00001,PPA,BiGG\n


In [24]:
len(ModelSEED_ID)

343679

In [25]:
ModelSEED_ID_unique = set(ModelSEED_ID)

In [26]:
ModelSEED_alias_df.to_csv('./data/modelSEED_alias.csv')

In [27]:
Source[15].strip() == 'KEGG'

True

In [28]:
KEGG_idx_ls = []
KEGG_ids = []
Modelseed_ids = []
Source_ids = []


for idx, src in tqdm(enumerate(Source)):
    if src.strip() == 'KEGG':
        KEGG_idx_ls.append(idx)
        
        Modelseed_ids.append(ModelSEED_ID[idx])
        KEGG_ids.append(External_ID[idx])
        Source_ids.append(Source[idx])
        
        

343679it [00:00, 1669335.07it/s]


In [29]:
len(Modelseed_ids)

14618

In [30]:
dict_modelseed_to_kegg = {}
dict_modelseed_to_kegg[ModelSEED_ID_header] = Modelseed_ids
dict_modelseed_to_kegg[External_ID_header]  = KEGG_ids
dict_modelseed_to_kegg[Source_header] = Source_ids

modelSEED_to_kegg_df = pd.DataFrame(dict_modelseed_to_kegg)

modelSEED_to_kegg_df.head()



,ModelSEED ID,External ID,Source\n
0,rxn00001,R00004,KEGG\n
1,rxn00002,R00005,KEGG\n
2,rxn00003,R00006,KEGG\n
3,rxn00004,R00008,KEGG\n
4,rxn00006,R00009,KEGG\n


In [32]:
equilibrator = pd.read_table('./data/MetaNetX_Reaction_Energies_equilibrator.tbl', header=None)

In [33]:
equilibrator_rxnid_mseeed = list(equilibrator[0])

In [34]:
KEGG_db_mseed = pd.read_table('./data/KEGG_Reactions_modelseed.tbl') 

In [35]:
KEGG_db_ids_mseed = list(KEGG_db_mseed['ID'])
KEGG_db_rxn_eqn_mseed = list(KEGG_db_mseed['EQUATION'])

In [36]:
KEGG_db_rxn_eqn_mseed_clean = []

for eqn in tqdm(KEGG_db_rxn_eqn_mseed):
    temp = eqn.replace('[0]', '').replace('(', '').replace(')', "")
    KEGG_db_rxn_eqn_mseed_clean.append(temp)
    

100%|██████████| 9722/9722 [00:00<00:00, 818947.29it/s]


In [37]:
KEGG_rxn_list_mseed = {}

for ix, ids in tqdm(enumerate(KEGG_db_ids_mseed)):
    KEGG_rxn_list_mseed[ids] = KEGG_db_rxn_eqn_mseed_clean[ix]

9722it [00:00, 1684583.31it/s]


## splitting data for parallel dG prediction- saves time

In [88]:
KEGG_db_ids_mseed_ar = np.array(KEGG_db_ids_mseed)
split_100_KEGG_ids_mseed = np.array_split(KEGG_db_ids_mseed_ar, 100)

KEGG_db_rxn_eqn_mseed_clean_ar = np.array(KEGG_db_rxn_eqn_mseed_clean)
split_100_KEGG_rxn_eqn_mseed_clean = np.array_split(KEGG_db_rxn_eqn_mseed_clean_ar, 100)



In [98]:
for i in range(100):
    id_ls = list(split_100_KEGG_ids_mseed[i])
    eqn_ls = list(split_100_KEGG_rxn_eqn_mseed_clean[i])
    
    temp_dict = {}
    for j, idis in enumerate(id_ls):
        temp_dict[idis] = eqn_ls[j]
        
        
    fname = './data/KEGG_split_rxn_data/KEGG_rxn_eqn_mseed_split_' + str(i)+'.json'
    
    with open(fname, 'w') as of:
        json.dump(temp_dict, of, indent=4)
    
    

## shell script to generate multiple run and job submission files

In [ ]:
!for i in {1..100}; do  cp subScript.pbs subScript_$i.pbs ; done

In [ ]:
!for i in {1..100}; do  cp db_bulk_dg_gen.py db_bulk_dg_gen_$i.py ; done

## bulk edit of model run python files for splits

In [150]:
for i in tqdm(range(100)):
    fread_name = 'db_bulk_dg_gen_' + str(i+1) + '.py'
    with open(fread_name, 'r') as f:
        temp_data = f.readlines()
        
#     print(temp_data[-7])
    temp_data[-3] = temp_data[-3].replace('0', str(i))
    temp_data[-7] = temp_data[-7].replace('0', str(i))
    
    with open(fread_name, 'w') as fl:
        fl.writelines(temp_data)

100%|██████████| 100/100 [00:00<00:00, 465.98it/s]


In [2]:
from tqdm import tqdm

for i in tqdm(range(100)):
    fread_name = './run_script/subScript_' + str(i+1) + '.pbs'
    with open(fread_name, 'r') as f:
        temp_data = f.readlines()
        
    temp_data[-5] = temp_data[-5].replace('0', str(i + 1))
    
    with open(fread_name, 'w') as fl:
        fl.writelines(temp_data)

100%|██████████| 100/100 [00:00<00:00, 385.26it/s]


## loading split data dG's to generate final file


In [48]:

for i in tqdm(range(100)):
    fname = './modelseed/dG_kegg_split_' + str(i) + '.csv'
    
    csv_read = pd.read_csv(fname)
    
    if i == 0:
        csv_final = csv_read
    else:
        csv_final = pd.concat([csv_final, csv_read])
    


100%|██████████| 100/100 [00:02<00:00, 40.33it/s]


In [113]:
csv_final.to_csv('./data/KEGG_dG_modelseed_ref.csv')

In [73]:
dG_kegg_id = list(csv_final['ID'])
dG_Kegg = list(csv_final['dG'])
dG_std_kegg = list(csv_final['std'])

dG_kegg_id_ar = np.array(csv_final['ID'])
dG_Kegg_ar = np.array(csv_final['dG'])
dG_std_kegg_ar = np.array(csv_final['std'])

In [75]:
Modelseed_ids_ar = np.array(Modelseed_ids)
KEGG_ids_ar = np.array(KEGG_ids)

In [93]:
list(np.where(Modelseed_ids_ar == 'rxn00013')[0])

[12]

In [101]:
KEGG_ids[Modelseed_ids.index('rxn00013')] 

'R00016'

In [97]:
A = json.load(open('./data/optstoic_v3_Sji_dict.json'))

False

In [103]:
mseed_dict_dup = {}
mseed_dict = {}
rid_not_inmseed_table = []


for i, mseedids in tqdm(enumerate(Modelseed_ids)):
    try:
        dG_val = dG_Kegg[dG_kegg_id.index(KEGG_ids[i])]
        std_val = dG_std_kegg[dG_kegg_id.index(KEGG_ids[i])]
    except: 
        dG_val = ''
        std_val = ''
        rid_not_inmseed_table.append(KEGG_ids[i])
        
    mseed_dict_keys = list(mseed_dict.keys())
    
    mseed_dict_dup[i] = {'ID': mseedids, 'KEGG': KEGG_ids[i], 'dG': dG_val, 'std': std_val}
    
    if mseedids in mseed_dict_keys:
        dG_dup = mseed_dict[mseedids]
        if dG_dup == np.NaN:
            mseed_dict[i] = {'ID': mseedids, 'dG': dG_val, 'std': std_val}
    else:
        mseed_dict[i] = {'ID': mseedids, 'dG': dG_val, 'std': std_val}
        

14618it [00:05, 2909.88it/s]


In [107]:
mseed_dG_df = pd.DataFrame(mseed_dict)

In [108]:
mseed_dG_df_t = mseed_dG_df.transpose()

In [114]:
mseed_dG_df_t.to_csv('./data/modelseed_first_dG.csv')

## Bulk estimation of dG for a list of KEGG reactions

In [53]:
KEGG_rxn_list = {"R00010" : "C01083 + C00001 <=> 2 C00031",
                 "R00303" : "C00092 + C00001 <=> C00031 + C00009",
                 "R00304" : "C00103 + C00001 <=> C00031 + C00009",
                 "R07294" : "C15524 + C00001 <=> C02137 + C00010",
                 "R01252" : "C00148 + C00026 + C00007 <=> C01157 + C00042 + C00011",
                 "R00406" : "C00091 + C00149 <=> C00042 + C04348",
                 "test": "C00026 + C00007 <=> C00042 + C00011"
                }



In [33]:
pH = 7 # any number between 0-14 
I = 0.1    #min_value=0.0, max_value=0.5)

for keys in KEGG_rxn_list:
    kegg_rxn_string = KEGG_rxn_list[keys]
    kegg_rxn_dict = parse_formula(kegg_rxn_string)
    mu, std, rule_df1, rule_df2 = get_dG0(kegg_rxn_dict, keys, pH, I, loaded_model, molsig_r1, molsig_r2, [], [], [])
    print(keys)
    print(kegg_rxn_string)
    print("dG = %.2f ± %.2f kJ/mol" % (mu, std))

R00010
C01083 + C00001 <=> 2 C00031
dG = -12.45 ± 3.49 kJ/mol
R00303
C00092 + C00001 <=> C00031 + C00009
dG = -12.40 ± 3.30 kJ/mol
R00304
C00103 + C00001 <=> C00031 + C00009
dG = -18.78 ± 3.37 kJ/mol
R07294
C15524 + C00001 <=> C02137 + C00010
dG = -14.46 ± 31.67 kJ/mol
R01252
C00148 + C00026 + C00007 <=> C01157 + C00042 + C00011
dG = -427.04 ± 44.08 kJ/mol
R00406
C00091 + C00149 <=> C00042 + C04348
dG = -3.27 ± 4.37 kJ/mol


In [148]:
pH = 7 # any number between 0-14 
I = 0.1    #min_value=0.0, max_value=0.5)

print('reached here...')

def bulk_dG_pred(Rxn_list, pH, I):
    print('started!!!')

    mu_ls = []
    std_ls = []
    kegg_id_ls = []
    for keys in tqdm(Rxn_list):
        kegg_rxn_string = Rxn_list[keys]
        kegg_rxn_dict = parse_formula(kegg_rxn_string)
        kegg_id_ls.append(keys)
        try: 
            mu, std, rule_df1, rule_df2 = get_dG0(kegg_rxn_dict, keys, pH, I, loaded_model, molsig_r1, molsig_r2, [], [], [])
            mu_ls.append(mu)
            std_ls.append(std)
        except: 
            mu_ls.append(np.NaN)
            std_ls.append(np.NaN)
    
    final_out_dict= {'ID': kegg_id_ls, 'dG': mu_ls, 'std': std_ls}
    output_df = pd.DataFrame(final_out_dict)
    return output_df    
    
        
        # print(keys)
        # print(kegg_rxn_string)
        # print("dG = %.2f ± %.2f kJ/mol" % (mu, std))
    

reached here...


['rxn00001',
 'rxn00002',
 'rxn00003',
 'rxn00004',
 'rxn00006',
 'rxn00007',
 'rxn00007',
 'rxn00008',
 'rxn00009',
 'rxn00010',
 'rxn00011',
 'rxn00012',
 'rxn00013',
 'rxn00014',
 'rxn00015',
 'rxn00016',
 'rxn00017',
 'rxn00018',
 'rxn00020',
 'rxn00020',
 'rxn00021',
 'rxn00022',
 'rxn00022',
 'rxn00023',
 'rxn00024',
 'rxn00025',
 'rxn00026',
 'rxn00027',
 'rxn00028',
 'rxn00029',
 'rxn00030',
 'rxn00031',
 'rxn00033',
 'rxn00034',
 'rxn00035',
 'rxn00036',
 'rxn00038',
 'rxn00039',
 'rxn00040',
 'rxn00041',
 'rxn00042',
 'rxn00043',
 'rxn00044',
 'rxn00045',
 'rxn00046',
 'rxn00047',
 'rxn00048',
 'rxn00051',
 'rxn00052',
 'rxn00054',
 'rxn00055',
 'rxn00056',
 'rxn00057',
 'rxn00058',
 'rxn00060',
 'rxn00061',
 'rxn00062',
 'rxn00063',
 'rxn00064',
 'rxn00065',
 'rxn00066',
 'rxn00067',
 'rxn00068',
 'rxn00069',
 'rxn00070',
 'rxn00071',
 'rxn00072',
 'rxn00074',
 'rxn00075',
 'rxn00076',
 'rxn00077',
 'rxn00078',
 'rxn00079',
 'rxn00080',
 'rxn00083',
 'rxn00084',
 'rxn00085',